In [1]:
import os
import sqlite3

os.chdir(r"C:\\Users\sivat\Desktop\Statistics\Python\DataQuest\Data_Analytics\Academy_awards")

conn = sqlite3.connect("nominations.db")


In [2]:
query1 = "pragma table_info(nominations);"
query2 = "select * from nominations limit 10;"

schema = conn.execute(query1).fetchall()
first_ten = conn.execute(query2).fetchall()

In [3]:
for r in schema:
    print(r)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'movie_names', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [4]:
for r in first_ten:
    print(r)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful ', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit ', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network ', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech ", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours ', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter ', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone ", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town ', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right ', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech ", 'Lionel Logue')


Creating ceremonies table.

In [5]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

table_query = "create table ceremonies(id integer PRIMARY KEY, year integer, host text);"
conn.execute(table_query)

OperationalError: table ceremonies already exists

Filling the ceremonies table with values.

In [6]:
insert_query = "INSERT INTO ceremonies (year, host) VALUES (?,?);"
conn.executemany(insert_query, years_hosts)

Testing the cermonies table.

In [7]:
conn.execute('select * from ceremonies limit 10').fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [8]:
conn.execute("pragma table_info(ceremonies);").fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'year', 'integer', 0, None, 0),
 (2, 'host', 'text', 0, None, 0)]

Turning on foreign keys to avoid non-existent foreign keys.

In [9]:
conn.execute("PRAGMA foreign_keys = ON;")
#conn.execute("drop table nominations2")

Since the year column cannot be removed from nominations table, a new table has to be created (nominations2), establish a relation with ceremonies, delete nominations table and rename nominations2 to nominations.

In [10]:
create_nominations_two = '''create table nominations2 
(id integer primary key, 
category text, 
nominee text, 
movie text, 
character text, 
won text,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));
'''

nom_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie_names as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''
joined_nominations = conn.execute(nom_query).fetchall()

conn.execute(create_nominations_two)

insert_nominations_two = '''insert into nominations2 (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''

conn.executemany(insert_nominations_two, joined_nominations)
print(conn.execute("select * from nominations2 limit 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful ', 'Uxbal', '0', 1), (2, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful ', 'Uxbal', '0', 12), (3, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit ', 'Rooster Cogburn', '0', 1), (4, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit ', 'Rooster Cogburn', '0', 12), (5, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network ', 'Mark Zuckerberg', '0', 1)]


Delete nominations table

In [11]:
conn.execute("drop table nominations;")

Rename nominations2 to nominations

In [12]:
conn.execute("alter table nominations2 rename to nominations;")

Creating movies, actors and movies_actors tables.

In [14]:
conn.execute("create table movies(id integer primary key, movie text);")

In [15]:
conn.execute("create table actors(id integer primary key, actor text);")

In [17]:
conn.execute('''create table movies_actors(id INTEGER PRIMARY KEY,movie_id INTEGER references movies(id), 
actor_id INTEGER references actors(id));''')

Populating actors and movies tables.

In [19]:
conn.execute("insert into actors(actor) select distinct nominee from nominations;")
conn.execute("insert into movies(movie) select distinct movie from nominations;")

In [20]:
print(conn.execute("select * from actors limit 5;").fetchall())
print(conn.execute("select * from movies limit 5;").fetchall())

[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]
[(1, 'Biutiful '), (2, 'True Grit '), (3, 'The Social Network '), (4, "The King's Speech "), (5, '127 Hours ')]


Populating the movis_actors join table.

In [21]:
movie_actor = conn.execute("select movie, nominee from nominations;").fetchall()

In [24]:
table_insert = '''insert into movies_actors(movie_id, actor_id) values ((select id from movies where movie == ?),
(select id from actors where actor == ?));'''
conn.executemany(table_insert, movie_actor)

In [25]:
print(conn.execute("select * from movies_actors limit 5;").fetchall())

[(1, 1, 1), (2, 1, 1), (3, 2, 2), (4, 2, 2), (5, 3, 3)]
